In [1]:
from bs4 import BeautifulSoup
import requests
from io import BytesIO
from datetime import datetime
from dateutil.parser import parse
import time
from selenium import webdriver
import undetected_chromedriver as uc 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By

from pprint import pprint
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd

In [2]:

# class to determine ganjil genap
def is_date(string, fuzzy=False):
    try: 
        parse(string, fuzzy=fuzzy)
        return True
    except ValueError:
        return False


In [3]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("SINOKOR")

In [4]:
s = slice(10)
bl_list

['SNKO073231100724 ',
 'SNKO073231101534 ',
 'SNKO073231100985',
 'SNKO073231200037',
 'SNKO073231200095',
 'SNKO073231101616 ',
 'SNKO073231100725 ',
 'SNKO073231101620',
 'SNKO073231101917',
 'SNKO073231201259',
 'SNKO073231101911 ',
 'SNKO073231200692',
 'SNKO073231201266 ',
 'SNKO073231201000 ',
 'SNKO073231201422 ',
 'SNKO073231200693',
 'SNKO073231201364 ',
 'SNKO073231101536 ']

In [5]:
def check_for_kw(word_list, input_string):
    for word in word_list:
        if word in input_string:
            return True
    return False

In [6]:
list_of_dict = []

dept_temp = None
arrival_temp = None
pickup_temp = None
return_temp = None

# web driver
options = Options()
options.add_argument("--window-size=1920,1280")
driver = uc.Chrome()

for i, bls in enumerate(bl_list):
    # acessing BL
    driver.get("http://ebiz.sinokor.co.kr/BLDetail?blno=" + bls)

    time.sleep(1)
    
    # getting containers in bl
    soup = BeautifulSoup(driver.page_source, 'lxml')

    # getting from to of the shipment
    places = soup.find_all('div', {'class', 'col-sm-6 col-md-6'}) 
    origin_city = places[0].text.replace('\n','').replace('  ','').split(" ")[0]
    origin_port = places[0].text.replace('\n','').replace('  ','').split(" ")[1]
    destination_city = places[1].text.replace('\n','').replace('  ','').split(" ")[0]
    destination_port = places[1].text.replace('\n','').replace('  ','').split(" ")[1]

    # list of containers in 1 bl
    container_table = soup.find_all('table', {'class', 'table detailTable dataTable no-footer'}) 
    list_of_containers = []
    for tag in container_table:
        for c, ctr  in enumerate(tag.find_all('tr', {"class":'odd'}) + tag.find_all('tr', {"class":'even'})):
            for c2, ctr2 in enumerate(ctr.find_all('td')):
                if len(ctr2.text) == 11:
                    list_of_containers.append(ctr2.text)
    
    # milestons of shipment
    milestone_table = soup.find_all('div', {'class', 'splitTable'})

    # appending milestones to list
    list_all = []
    for tag2 in milestone_table:
        for m1, milestone1  in enumerate(tag2.find_all('tr')):
            for m2, theader in enumerate(milestone1.find_all('th',{'class', 'firstTh'})):
                list_all.append(theader.text)
            for m3, milestone2 in enumerate(milestone1.find_all('td')):
                list_all.append(milestone2.text)
    
    # splitting list
    for li, element in enumerate(list_all):
        try:
            if element == "Departure":
                dept_temp = li
            elif element == "Arrival":
                arrival_temp = li
            elif element.split(" ")[0] == "Pickup":
                pickup_temp = li
            elif element.split(" ")[0] == "Return":
                return_temp = li
        except Exception as e:
            print(e)
            
    # list/movement
    try:
        departure_milestone = list_all[dept_temp:arrival_temp]
        arrival_milestone = list_all[arrival_temp:pickup_temp]
        pickup_milestone = list_all[pickup_temp:return_temp]
        return_milestone = list_all[return_temp:]
    except Exception as e:
            print(e)
    
    # pickup and return dictionaries
    pickup_dict = {}
    for p, pickup_date in enumerate(pickup_milestone):
        if is_date(pickup_date):
            case_pickup = {pickup_milestone[p-2]:pickup_date}
            pickup_dict.update(case_pickup)

    return_dict = {}
    for r, return_date in enumerate(return_milestone):
        if is_date(return_date):
            case_return = {return_milestone[r-2]:return_date}
            return_dict.update(case_return)

    for c, container in enumerate(list_of_containers):
        current_dict = {}
        try:
            current_dict.update({"Liners":"SINOKOR"})
            current_dict.update({"BL Number":bls})
            current_dict.update({"Container Number":container})
            current_dict.update({"From": origin_city})
            current_dict.update({"To": destination_city})
            current_dict.update({"ATD":departure_milestone[3][s]})
            current_dict.update({"ATA":arrival_milestone[3][s]})
            current_dict.update({"Container Release":pickup_dict[container][s]})
            current_dict.update({"Container Return":return_dict[container][s]})
        except:
            pass

        list_of_dict.append(current_dict)


In [7]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict)

Today's Collection Name ===>  all_tracking_Jan-10-2024
Today's Collection Has Been Made
Inserting Many Complete!!


In [8]:
list_of_dict

[{'Liners': 'SINOKOR',
  'BL Number': 'SNKO073231100724 ',
  'Container Number': 'BMOU6138318',
  'From': 'Jakarta',
  'To': 'Shanghai',
  'ATD': '2023-12-14',
  'ATA': '2023-12-31',
  'Container Release': '2024-01-03',
  'Container Return': '2024-01-03',
  '_id': ObjectId('659ca417ab7ae1881c926c8e')},
 {'Liners': 'SINOKOR',
  'BL Number': 'SNKO073231100724 ',
  'Container Number': 'HALU5676241',
  'From': 'Jakarta',
  'To': 'Shanghai',
  'ATD': '2023-12-14',
  'ATA': '2023-12-31',
  'Container Release': '2024-01-03',
  'Container Return': '2024-01-03',
  '_id': ObjectId('659ca417ab7ae1881c926c8f')},
 {'Liners': 'SINOKOR',
  'BL Number': 'SNKO073231100724 ',
  'Container Number': 'SEKU4982086',
  'From': 'Jakarta',
  'To': 'Shanghai',
  'ATD': '2023-12-14',
  'ATA': '2023-12-31',
  'Container Release': '2024-01-02',
  'Container Return': '2024-01-02',
  '_id': ObjectId('659ca417ab7ae1881c926c90')},
 {'Liners': 'SINOKOR',
  'BL Number': 'SNKO073231100724 ',
  'Container Number': 'SKHU92

In [9]:
import datetime
df = pd.DataFrame(list_of_dict)

excel_file_path = 'export excel/SINOKOR.xlsx'

df.to_excel(excel_file_path, index=False)

In [10]:
gagal

NameError: name 'gagal' is not defined